In [1]:
from hough_functions import LineDetector
from ultralytics import YOLO
import csv
import cv2

In [2]:
def calculate_x_val(line, y):
    '''
    :param line: Coordinates [[x1, y1, x2, y2]] that define a line.
    :param y: Y value of the bottom of a detected object.
    :return: X value where object intercepts the line.
    '''
    x1 = line[0][0]
    y1 = line[0][1]
    x2 = line[0][2]
    y2 = line[0][3]
    slope = (y2 - y1) / (x2 - x1)
    x = (y - y1)/slope + x1
    return x

def calculate_distance(w0, w1, d0):
    '''
    :param w0: Width of the track in pixels at the edge of the screen (bottom)
    :param w1: Width of the track in pixels at the bottom of the detected object
    :param d1: Distance from the camera to w0 (in Feet)
    :return: Distance to w1
    '''
    return d0 * (w0/w1)

In [3]:
##########################

# Detect objects and lines

##########################

image = cv2.imread('car_track.png')
model = YOLO('yolov8n.pt')
results = model('car_track.png')
detector = LineDetector()
lines = detector.detect_lines_image('car_track.png')


image 1/1 /home/matteo/Projects/Intramotev/YOLOv8_Hough_Combine/car_track.png: 384x640 1 train, 49.0ms
Speed: 1.8ms preprocess, 49.0ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)
Line Detected with equation x = (y - 846) / -1.7313432835820894 + 479
Line Detected with equation x = (y - 424) / 1.8091286307053942 + 814


In [4]:
##########################

# Write to CSV File

##########################

detected_objects = results[0].boxes
csv_file_name = 'detected_objects.csv'
object_names = results[0].names
with open(csv_file_name, mode='w', newline='') as file:
    writer = csv.writer(file)
    
    writer.writerow(['Class ID', 'Object Name','Confidence', 'X1', 'Y1', 'X2', 'Y2'])

    for box in detected_objects:
        class_id = box.cls[0].item()
        conf = box.conf[0].item()
        cords = box.xyxy[0].tolist()  # formatted as [x1, y1, x2, y2]
        object_name = object_names.get(class_id, 'Unknown')
        
        # Write the object data to the CSV file
        writer.writerow([class_id, object_name, conf, *cords])
        
##########################

# Calculate Distance

##########################
        
bottom_object = cords[3]
x1 = calculate_x_val(lines[0], 580)
x2 = calculate_x_val(lines[1], 580)

distance = calculate_distance(580, (x2-x1), 5)

In [5]:
##########################

# Display Picture

##########################

detected_objects = results[0].boxes
object_names = results[0].names

# Draw bounding boxes on the image
for box in detected_objects:
    # Extract box coordinates
    x1, y1, x2, y2 = box.xyxy[0].tolist()
    # Convert coordinates to integers
    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
    # Get class ID and confidence
    class_id = box.cls[0].item()
    conf = box.conf[0].item()
    # Get object name
    object_name = object_names.get(class_id, 'Unknown')
    # Draw bounding box on the image
    cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
    # Put text showing class name and confidence
    cv2.putText(image, f'{object_name}: {conf:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    cv2.putText(image, f'Distance: {distance:.2f}', (x1, y2 + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
cv2.imwrite('processed_image.jpg', image)

True